# Grundkompetenz Datenbanken

## Setup

Hier werden die benötigten Daten von der Konfigurationsdatei abgelesen, um eine Datenbankverbindung herzustellen.

In [1]:
from pymongo import MongoClient
import configparser
import pprint
jsonp = pprint.pprint

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

db_username = config['Database']['USER']
db_password = config['Database']['PASS']
db_hostname = config['Database']['HOST']

Hier wird die Datenbankverbindung hergestellt.

In [3]:
client = MongoClient("mongodb+srv://{USERNAME}:{PASSWORD}@{HOSTNAME}".format(USERNAME = db_username, 
                                                                             PASSWORD = db_password, 
                                                                             HOSTNAME = db_hostname))
db = client.sample_mflix

Testabfrage:

In [4]:
print(db.list_collection_names())

['sessions', 'movies', 'users', 'comments', 'theaters']


5 Users ausgeben

In [ ]:
users = db.users.find({})[:5]
for user in users:
    jsonp(user)

Kommentare ausgeben

In [ ]:
comments = db.comments.find({})[:5]
for comment in comments:
    jsonp(comment)

find() Nach einer bestimmten Bedingung ausgeben

In [ ]:
users2 = db.users.find({'name':"Cersei Lannister"})
for user in users2:
    jsonp(user)

In [ ]:
query = {'title'}
movie = db.movies.find(query)
for film in movie:
    jsonp(film)

find() mit "$gt" und "$lt" abfrage 
"$gt" => greater than
"$lt" => lower than

In [ ]:
yearquery = {'year':{"$lt":1893}}
movie2 = db.movies.find(yearquery)[:3]
for film in movie2:
    jsonp(film)

find() mit "$regex" Abfrage nach einer Zeichenkette

titlequery = {'title':{"$regex":"^s"}}
movie3 = db.movies.find(titlequery)[:2]
for film in movie3:
    jsonp(film)

In [ ]:
genrequery = {'genres':{"$regex":"^Drama"}}
movie4 = db.movies.find(genrequery)[:2]
for film in movie4:
    jsonp(film)

find() Meherere User nach mehreren Bedingungen ausgeben

In [ ]:
users3 = db.users.find({"$or":[{'name':"Cersei Lannister"},{'name':"Catelyn Stark"}]})
for user in users3:
    jsonp(user)

find() mit sort() verknuepfen
-1 = absteigend
1  = aufsteigend

In [ ]:
movie5 = db.movies.find().sort('runtime', -1)[:5] 
for film in movie5:
    jsonp(film)

Mehere Abfragen gleichzeitig

In [ ]:
movie6 = db.movies.find({ 'year': {'$gt': 2007}, '$and': [{'genres': 'Drama'},{'type': 'series'}]})[:2]
for film in movie6:
    jsonp(film)

Alle genres von Movie abfragen mit distinct()

In [ ]:
allgenres = db.movies.distinct('genres')
print(allgenres)

In [ ]:
query = {'awards.nominations': {'$gt': 1}}
nomination = db.movies.find(query)
for num in nomination:
    print(num['awards']['nominations'])

In [ ]:
titles = db.movies.find({ 'year': {'$gt': 2013}, '$and': [{'genres': 'Sci-Fi'},{'type': 'movie'},{'awards.wins': {'$gt': 44}}]})
for film in titles:
    jsonp(film['title'])

Join of movies and comments

In [ ]:
allmovies = db.movies.find({})
allcomments = db.comments.find({})

print(join(allmovies, allcomments, "_id", "movie_id"))

Sammlung Komplexer Fragestellungen
Welche Filme erhalten die meisten Kommentare? (Schauspieler, Genres etc..)
Welche Filme haben die meisten Kommentaren?
Welche Filme mit 10 awards Gewinnen haben die meisten Kommentar

In [ ]:
test = db.movies.aggregate([
    {"$group" : {"_id":{"_id":"$movie_id"}, "count":{"$sum":1}}}
])
for test2 in test:
    jsonp(test2)

# Fragen zu den Abfragen

### Welche Datenbank hast Du gewählt? 

Wir haben uns für MongoDB entschieden.

### Warum hast Du diese gewählt?

Wir haben uns für MongoDB entschieden weil MongoDB viele gut strukturierte Testdaten bereitstellt. Wir haben bis jetzt noch nie mit Dokumentdatenbanken gearbeitet und wollten somit unsere Kompetenzen erweitern. 

### Wie sehen die Daten aus?

Die Daten werden von MongoDB als json Strings zurückgegeben.
Die Tabellen heissen unter MongoDB "Collections" und unterscheiden sich von herkömmlichen Tabellen in relationalen Datenbanken, indem sie kein Datenstrakturschema vorgeben.

### Inwiefern ist das besser als eine relationale Datenbank?

Da die Daten kein Schema haben, können Änderungen bei der Datenerfassung leichter umgesetzt werden. Für grosse Datenservices lohnt sich MongoDB, da die Datenbank horizontal skalierbar ist (mehrere Nodes) und die Latenz und Auslastung weltweit minimiert werden kann. Bei objektorientiertem Code kann ein Objekt zusätzlich schnell geparst und weiterverarbeitet werden.

### Wie sehen komplexe Fragestellungen (Abfrage) zu den Daten aus, und warum sind sie komplex? Vergleiche es mit eine SQL, würde es komplizierter sein, welche Vorteile gibt es gegenüber SQL?

Bei komplexen Abfragen werden die Daten nicht nur abgelesen, sondern stark verarbeitet (Datenaufbereitung), bereinigt, gefiltert und mit anderen Daten kombiniert.

# Komplexe Abfragen

Anzahl Kommentare aller Filme zurückgeben, die mindestens 150 Awards gewonnen haben.

In [5]:
result = db.movies.aggregate([
    {
        '$addFields': {
            'movie_id': {
                '$toString': '$_id'
            }
        }
    }, {
        '$lookup': {
            'from': 'comments', 
            'localField': '_id', 
            'foreignField': 'movie_id', 
            'as': 'comments'
        }
    }, {
        '$addFields': {
            'countComments': {
                '$size': '$comments'
            }
        }
    }, {
        '$match': {
            'countComments': {
                '$gt': 0
            },
            'awards.wins': {
                '$gt': 149
            }
        }
    }, {
        '$project': {
            'countComments': 1,
            'awards.wins': 1,
            'title': 1, 
            '_id': 0
        }
    }, {
        '$sort': {
            'awards.wins': -1
        }
    }
])

for var in result:
    jsonp(var)

{'awards': {'wins': 267}, 'countComments': 1, 'title': '12 Years a Slave'}
{'awards': {'wins': 231}, 'countComments': 1, 'title': 'Gravity'}
{'awards': {'wins': 185}, 'countComments': 1, 'title': 'Boyhood'}
{'awards': {'wins': 175},
 'countComments': 133,
 'title': 'The Lord of the Rings: The Return of the King'}
{'awards': {'wins': 172}, 'countComments': 1, 'title': 'No Country for Old Men'}
{'awards': {'wins': 171}, 'countComments': 1, 'title': 'The Social Network'}
{'awards': {'wins': 162}, 'countComments': 1, 'title': 'Inception'}
{'awards': {'wins': 161}, 'countComments': 127, 'title': 'Slumdog Millionaire'}
{'awards': {'wins': 161}, 'countComments': 1, 'title': 'The Artist'}


Welche 10 User haben die meisten Kommentare geschrieben?

In [6]:
result = db.comments.aggregate([
    {
        '$group': {
            '_id': '$email', 
            'countComments': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'countComments': -1
        }
    }, {
        '$limit': 10
    }, {
        '$lookup': {
            'from': 'users', 
            'localField': '_id', 
            'foreignField': 'email', 
            'as': 'userinfo'
        }
    }, {
        '$project': {
            'userinfo.name': 1, 
            'userinfo.email': 1, 
            'countComments': 1, 
            '_id': 0
        }
    }
])

for var in result:
    jsonp(var)

{'countComments': 277,
 'userinfo': [{'email': 'roger_ashton-griffiths@gameofthron.es',
               'name': 'Mace Tyrell'}]}
{'countComments': 260,
 'userinfo': [{'email': 'jonathan_pryce@gameofthron.es',
               'name': 'The High Sparrow'}]}
{'countComments': 260,
 'userinfo': [{'email': 'ron_donachie@gameofthron.es',
               'name': 'Rodrik Cassel'}]}
{'countComments': 258,
 'userinfo': [{'email': 'nathalie_emmanuel@gameofthron.es',
               'name': 'Missandei'}]}
{'countComments': 257,
 'userinfo': [{'email': 'robert_jordan@fakegmail.com',
               'name': 'Robert Jordan'}]}
{'countComments': 251,
 'userinfo': [{'email': 'paul_kaye@gameofthron.es', 'name': 'Thoros of Myr'}]}
{'countComments': 251,
 'userinfo': [{'email': 'sophie_turner@gameofthron.es', 'name': 'Sansa Stark'}]}
{'countComments': 248,
 'userinfo': [{'email': 'donna_smith@fakegmail.com', 'name': 'Donna Smith'}]}
{'countComments': 248,
 'userinfo': [{'email': 'nicholas_johnson@fakegmail.com'

Über welchen 10 Filme wurde am meisten kommentiert?

In [7]:
result = db.comments.aggregate([
    {
        '$group': {
            '_id': '$movie_id', 
            'countComments': {
                '$sum': 1
            }
        }
    }, {
        '$lookup': {
            'from': 'movies', 
            'localField': '_id', 
            'foreignField': '_id', 
            'as': 'movie'
        }
    }, {
        '$sort': {
            'countComments': -1
        }
    }, {
        '$limit': 10
    }, {
        '$project': {
            'countComments': 1, 
            'movie.title': 1,
            '_id': 0
        }
    }
])

for var in result:
    jsonp(var)

{'countComments': 161, 'movie': [{'title': 'The Taking of Pelham 1 2 3'}]}
{'countComments': 158, 'movie': [{'title': 'About a Boy'}]}
{'countComments': 158, 'movie': [{'title': 'Terminator Salvation'}]}
{'countComments': 158, 'movie': [{'title': "Ocean's Eleven"}]}
{'countComments': 158, 'movie': [{'title': '50 First Dates'}]}
{'countComments': 157, 'movie': [{'title': 'Sherlock Holmes'}]}
{'countComments': 157, 'movie': [{'title': 'The Mummy'}]}
{'countComments': 155, 'movie': [{'title': 'Hellboy II: The Golden Army'}]}
{'countComments': 154,
 'movie': [{'title': 'Anchorman: The Legend of Ron Burgundy'}]}
{'countComments': 154, 'movie': [{'title': 'The Mummy Returns'}]}


Welcher Staat hat die meisten "Theaters"?

In [8]:
result = db.theaters.aggregate([
    {
        '$addFields': {
            'state': '$location.address.state'
        }
    }, {
        '$group': {
            '_id': '$location.address.state', 
            'countTheaters': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'countTheaters': -1
        }
    }, {
        '$limit': 10
    }
])

for var in result:
    jsonp(var)

{'_id': 'CA', 'countTheaters': 169}
{'_id': 'TX', 'countTheaters': 160}
{'_id': 'FL', 'countTheaters': 111}
{'_id': 'NY', 'countTheaters': 81}
{'_id': 'IL', 'countTheaters': 70}
{'_id': 'PA', 'countTheaters': 55}
{'_id': 'OH', 'countTheaters': 52}
{'_id': 'VA', 'countTheaters': 45}
{'_id': 'GA', 'countTheaters': 45}
{'_id': 'MI', 'countTheaters': 45}


Welche Stadt hat die meisten "Theaters" und in welchem Staat befindet sich die Stadt?

In [9]:
result = db.theaters.aggregate([
    {
        '$addFields': {
            'city': '$location.address.city'
        }
    }, {
        '$group': {
            '_id': '$city', 
            'countTheaters': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'countTheaters': -1
        }
    }, {
        '$lookup': {
            'from': 'theaters', 
            'localField': '_id', 
            'foreignField': 'location.address.city', 
            'as': 'theaters'
        }
    }, {
        '$addFields': {
            'state': {
                '$first': '$theaters.location.address.state'
            }
        }
    }, {
        '$addFields': {
            'city': '$_id'
        }
    }, {
        '$project': {
            'theaters': 0, 
            '_id': 0
        }
    }, {
        '$limit': 10
    }
])

for var in result:
    jsonp(var)

{'city': 'Las Vegas', 'countTheaters': 29, 'state': 'NV'}
{'city': 'Houston', 'countTheaters': 22, 'state': 'TX'}
{'city': 'San Antonio', 'countTheaters': 14, 'state': 'TX'}
{'city': 'Orlando', 'countTheaters': 13, 'state': 'FL'}
{'city': 'Dallas', 'countTheaters': 12, 'state': 'TX'}
{'city': 'Los Angeles', 'countTheaters': 12, 'state': 'CA'}
{'city': 'Atlanta', 'countTheaters': 10, 'state': 'GA'}
{'city': 'San Francisco', 'countTheaters': 9, 'state': 'CA'}
{'city': 'Jacksonville', 'countTheaters': 9, 'state': 'FL'}
{'city': 'Miami', 'countTheaters': 8, 'state': 'FL'}
